In [19]:
import tensorflow as tf
import keras
#from keras.backend.tensorflow_backend import set_session
#from tensorflow.python.keras import backend as K


import numpy as np
from sklearn.model_selection import train_test_split
import os, glob
import matplotlib.pyplot as plt
import cv2
from keras.applications.mobilenet_v2 import MobileNetV2

from DataGenerator import DataGenerator


config = tf.compat.v1.ConfigProto()
#config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = False
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

In [ ]:
base_path = '/mnt2/iris_processed/'

json_list = glob.glob(os.path.join(base_path, '*/data/*.json'))

json_list[:10]

In [ ]:
# np.savez_compressed(os.path.join(base_path, 'json_list.npz'), data=json_list)

base_path = '/mnt2/iris_processed/'
json_list = np.load(os.path.join(base_path, 'json_list.npz'))['data']

In [ ]:
print(len(json_list))

In [ ]:
json_list_train, json_list_val = train_test_split(json_list, test_size=0.1, random_state=2020)

print(len(json_list_train), len(json_list_val))

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = 96
N_CHANNEL = 3

train_gen = DataGenerator(
    phase='train',
    file_list=json_list_train,
    batch_size=BATCH_SIZE,
    img_size=IMG_SIZE,
    n_channel=N_CHANNEL
)

val_gen = DataGenerator(
    phase='val',
    file_list=json_list_val,
    batch_size=BATCH_SIZE,
    img_size=IMG_SIZE,
    n_channel=N_CHANNEL
)

print("Done!")

In [ ]:
test_data = val_gen.__getitem__(0)

print(test_data[0].shape)
print(test_data[1].shape)

test_img = test_data[0][0].squeeze()
test_contour = test_data[1][0].squeeze().reshape((-1, 2))

print(test_contour)

for p in test_contour:
    cv2.circle(test_img, center=tuple(p.astype(int)), radius=1, color=1, thickness=-1, lineType=cv2.LINE_AA)

plt.imshow(test_img)

In [ ]:
from keras.layers import GlobalAveragePooling2D, BatchNormalization, Dense
from keras.models import Model

base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, N_CHANNEL))

x = base_model.output
x = GlobalAveragePooling2D()(x)

x = BatchNormalization()(x)
x = Dense(1280, activation='relu')(x)

x = BatchNormalization()(x)
output = Dense(38, activation='linear')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(loss='mse', optimizer='adam')

model.summary()

In [ ]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from datetime import datetime

os.makedirs('models', exist_ok=True)
start_time = datetiem.now().strftim('%Y-%m-%d_%H-%M-%S')

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=50,
    callbacks=[
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, verbose=1, mode='auto', min_lr=1e-05),
        ModelCheckpoint('models/%s-{epoch:02d}-{val_loss:.2f}.h5' % (start_time), monitor='val_loss', save_best_only=True, mode='min', verbose=1),
        TensorBoard(log_dir='logs/%s' % (start_time))
    ],
    use_multiprocessing=True,
    workers=5,
    max_queue_size=5
)

In [ ]:
from keras.models import load_model
best_model = load_model('models/2020-08-13_09-15-19-08-19.74.h5')  # 모델 결과 따라 변경할 것.

In [ ]:
test_data = val_gen.__getitem__(0)

fig, axes = plt.subplots(BATCH_SIZE, 1, figsize=(10, 100))

for i, (img, contour) in enumerate(zip(test_data[0], test_data[1])):
    # ground truth
    gt_contour = contour.reshape((-1, 2))
    
    for p in gt_contour:
        cv2.circle(img, center=tuple(p.astype(int)), radius=1, color=(1, 0, 0), thickness=-1, lineType=cv2.LINE_AA)
        
    # prediction
    pred_contour = best_model.predict(np.expand_dims(img, axis=0)).reshape((-1, 2))
    
    for p in pred_contour:
        cv2.circle(img, center=tuple(p.astype(int)), radius=1, color=(0, 0, 1), thickness=-1, lineType=cv2.LINE_AA)
    
    axes[i].imshow(img)